In [34]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from torch import nn

# Set the hyperparameters for data creation
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

# 1. Create multi-class data
X_blob, y_blob = make_blobs(n_samples=1000,
    n_features=NUM_FEATURES, # X features
    centers=NUM_CLASSES, # y labels
    cluster_std=1.5, # give the clusters a little shake up (try changing this to 1.0, the default)
    random_state=RANDOM_SEED
)

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X_blob, y_blob,
                                                 random_state=42,
                                                 test_size=0.2)

In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [37]:
X_train,X_test = torch.from_numpy(X_train).type(torch.float).to(device),torch.from_numpy(X_test).type(torch.float).to(device)
y_train,y_test = torch.from_numpy(y_train).type(torch.LongTensor).to(device),torch.from_numpy(y_test).type(torch.LongTensor).to(device)

In [38]:
X_train.dtype,X_test.dtype

(torch.float32, torch.float32)

In [39]:
class MultiClass(nn.Module):
  def __init__(self, in_features,out_features):
    super().__init__()
    self.in_features  = in_features
    self.out_features = out_features



    self.sequence = nn.Sequential(

      nn.Linear(in_features=self.in_features, out_features=16),
      nn.ReLU(),
      nn.Linear(in_features=16, out_features=32),
      nn.ReLU(),
      nn.Linear(in_features=32, out_features=self.out_features)

    )

  def forward(self,x):
    return self.sequence(x)



In [40]:
model = MultiClass(2,4)
model

MultiClass(
  (sequence): Sequential(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [41]:
loss_fn   = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.01)

In [42]:
def accuracy(y_pred,y_true):
  return(torch.eq(y_pred,y_true).sum().item()/len(y_pred))*100

In [44]:
torch.manual_seed(42)

Epochs = 100

for epoch in range(Epochs):
  model.train()
  optimizer.zero_grad()

  pred_logit = model(X_train)
  y_pred = torch.argmax(torch.softmax(pred_logit,dim=1),dim=1)

  loss = loss_fn(pred_logit,y_train)

  acc = accuracy(y_pred,y_train)

  model.eval()
  with torch.inference_mode():
    y_test_logit = model(X_test)
    y_test_pred = torch.argmax(torch.softmax(y_test_logit,dim=1),dim=1)
    loss_test = loss_fn(y_test_logit,y_test)
    acc_test = accuracy(y_test_pred,y_test)

  if epoch %10==0:

    print(f"Train Loss: {loss:0.3f}, Train Accuracy: {acc:0.3f} | Test Loss: {loss_test:0.3f}, Test Accuracy: {acc_test:0.3f}")



Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
Train Loss: 1.361, Train Accuracy: 45.500 | Test Loss: 1.365, Test Accuracy: 44.500
